In [1]:
import torch
from torch.nn import functional as F
import pickle
from tqdm import tqdm

In [2]:
width, height = 10, 10

In [3]:
empty_board = torch.zeros(2, width, height)
empty_board

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.,

In [4]:
def get_next_state(state, move: int, width=10, height=10):
    # only return but not change the current state
    state_next = state.clone()
    move_x = move // width
    move_y = move % height
    state_next[0, move_x, move_y] = 1
    state_next[0], state_next[1] = torch.tensor(state_next[1].tolist()), torch.tensor(state_next[0].tolist())
    return state_next

In [5]:
traces = [
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	75,	46,	47,	37,	77,	28],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	75,	46,	47,	37,	77,	73],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	46,	34,	57,	35,	33,	23,	12,	53,	26,	62,	71,	42,	52,	75,	86,	31],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	46,	34,	57,	35,	33,	23,	12,	53,	26,	62,	71,	42,	52,	75,	31,	86],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	46,	34,	57,	35,	33,	23,	12,	53,	26,	62,	71,	42,	75,	52,	72,	22],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	46,	34,	57,	35,	33,	23,	12,	53,	62,	26,	17,	75,	42,	86,	97,	41,	14,	15,	37,	25],
    [   44,	54,	45,	43,	55,	65,	32,	76,	87,	66,	56,	67,	64,	85,	94,	58,	49,	68,	69,	78,	48,	88,	98,	46,	34,	57,	35,	33,	23,	12,	53,	62,	26,	17,	75,	42,	86,	97,	41,	14,	15,	25,	37,	59,	14],
    [   55,	45,	44,	46,	33,	66,	56,	54,	36,	37,	35,	34,	53,	26,	43,	23,	24,	12,	1,	42,	32,	84,	75,	64,	74,	65,	63,	73,	62,	77,	85,	52,	25,	47,	67,	48,	49,	37,	15,	17,	27,	81,	82,	61,	51,	72,	76,	77,	58],
    [   55,	45,	44,	46,	33,	66,	56,	54,	36,	37,	35,	34,	53,	26,	43,	23,	24,	12,	1,	42,	32,	84,	75,	64,	74,	65,	63,	73,	62,	77,	85,	52,	25,	47,	67,	48,	49,	37,	15,	17,	27,	81,	82,	61,	51,	72,	76,	58,	78,	77,	38,	5,	22,	11],
    [   55,	45,	44,	46,	33,	66,	56,	54,	36,	37,	35,	34,	53,	26,	43,	23,	24,	12,	1,	42,	32,	84,	75,	64,	74,	65,	63,	73,	62,	77,	85,	52,	25,	47,	67,	48,	49,	37,	15,	17,	27,	81,	82,	61,	51,	72,	76,	58,	77,	78,	22,	11,	38,	68,	88,	5],
    [   55,	45,	44,	46,	33,	66,	56,	54,	36,	37,	35,	34,	53,	26,	43,	23,	24,	12,	1,	42,	32,	84,	75,	64,	74,	65,	63,	73,	62,	77,	85,	52,	25,	47,	67,	48,	49,	37,	15,	17,	27,	81,	82,	61,	51,	72,	76,	58,	77,	78,	22,	11,	16,	68,	38,	88]
]
draw = [False, False, False, False, False, False, False, False, True, True, False]
dupl = [0, 28, 23, 37, 35, 31, 41, 0, 47, 48, 52]

In [6]:
def gen_game(start_board, trace, draw, dupl):
    game = []
    board = start_board
    for move in trace:
        step = [
            board.clone(), 
            # torch.tensor(move, dtype=torch.float32), 
            F.softmax(
                torch.tensor(
                    F.one_hot(torch.tensor(move).clone().detach(), num_classes=width*height), dtype=torch.float32
                )*8, # soft but not that soft
                dim=0
            ),
            torch.tensor(0, dtype=torch.float32)]
        board = get_next_state(board, move)
        game.append(step)
    if not draw:
        r = 1 # last player observes the board, moves, and makes a win
        for reverse_g in range(len(game)-1, -1, -1):
            game[reverse_g][2] = torch.tensor(r, dtype=torch.float32)
            # invert the game result for the previous player
            r = -r
    return game[dupl:], board

In [7]:
games = []  
for i in range(len(traces)):
    game, board = gen_game(empty_board, traces[i], draw[i], dupl[i])
    games += game

/tmp/ipykernel_6585/4203176346.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(


In [8]:
len(games), games[0]

(136,
 [tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
  
          [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
           [0., 0., 0., 0., 0., 0., 

In [16]:
def augment_game(games):
    game_aug = []
    # geometric augmentation
    for step in games:
        game_aug.append(step)
        # rotate 90, 180, 270; flip left-right, up-down; transpose
        # rotate 90 degrees
        game_aug.append([
            torch.rot90(step[0], 1, [1, 2]),
            step[1].reshape(width, height).rot90(1, [0, 1]).reshape(-1),
            step[2]
        ])
        # rotate 180 degrees
        game_aug.append([
            torch.rot90(step[0], 2, [1, 2]),
            step[1].reshape(width, height).rot90(2, [0, 1]).reshape(-1),
            step[2]
        ])
        # rotate 270 degrees
        game_aug.append([
            torch.rot90(step[0], 3, [1, 2]),
            step[1].reshape(width, height).rot90(3, [0, 1]).reshape(-1),
            step[2]
        ])
        # flip left-right
        game_aug.append([
            torch.flip(step[0], [2]),
            step[1].reshape(width, height).flip(1).reshape(-1),
            step[2]
        ])
        # flip up-down
        game_aug.append([
            torch.flip(step[0], [1]),
            step[1].reshape(width, height).flip(0).reshape(-1),
            step[2]
        ])
        # transpose
        game_aug.append([
            step[0].transpose(1,2),
            step[1].reshape(width, height).transpose(0,1).reshape(-1),
            step[2]
        ])
    # noise augmentation
    game_aug_aug = []
    for step in tqdm(game_aug):
        game_aug_aug.append(step)
        # add noise to the board by flipping some pieces
        for aug_step in range(5):
            step_aug = step[0].clone()
            # randomly flip 2% of the pieces from the board
            target = torch.randperm(width*height)[:int(width*height*0.02)]
            step_aug[0].reshape(-1)[target] = 1 - step_aug[0].reshape(-1)[target]
            step_aug[1].reshape(-1)[target] = 1 - step_aug[1].reshape(-1)[target]
            # append the augmented step
            game_aug_aug.append([
                step_aug,
                step[1],
                step[2]
            ])
    return game_aug_aug

In [17]:
games_aug = augment_game(games)

100%|██████████| 952/952 [00:00<00:00, 2508.36it/s]


In [18]:
len(games_aug)

5712

In [19]:
# save the games
pretrain_games = {
    'board': [g[0] for g in games],
    'move': [g[1] for g in games],
    'result': [g[2] for g in games]
}
with open('games.pickle', 'wb') as f:
    pickle.dump(pretrain_games, f)

In [20]:
# check the saved games
with open('games.pickle', 'rb') as f:
    pretrain_games = pickle.load(f)
pretrain_games['board'][0], pretrain_games['move'][0], pretrain_games['result'][0]

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         

In [21]:
# save the augmented games
pretrain_games_aug = {
    'board': [g[0] for g in games_aug],
    'move': [g[1] for g in games_aug],
    'result': [g[2] for g in games_aug]
}
with open('games_aug.pickle', 'wb') as f:
    pickle.dump(pretrain_games_aug, f)

In [22]:
# check the saved augmented games
with open('games_aug.pickle', 'rb') as f:
    pretrain_games_aug = pickle.load(f)
pretrain_games_aug['board'][0], pretrain_games_aug['move'][0], pretrain_games_aug['result'][0]

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         